## Resumen general del código

En este código se realiza el proceso de Georreferenciación de las direcciones incluidas en el insumo inicial. Este código devuelve un archivo con las direcciones, su localización en términos de latitud y longitud, y una calificación de esta geolocalización, de forma que se pueden descartar aquellos casos donde el proceso no fue exitoso.

Los pasos de ejecución son los siguientes:

- Ejecución de las librerías requeridas.
- Lectura de las tablas requeridas para el análisis a realizar.
- Se debe garantizar la existencia de las tablas en las rutas que se especifican en el programa.
- Procesamiento, limpieza y alistamiento sobre el campo de direcciones.
- Ejecución de la función geocoder_igac(.) sobre las direcciones procesadas.
- Export de resultados

In [ ]:
import requests
import numpy as np
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import geopy.geocoders as geocoders
import pandas as pd
import geocoder
from random import randint
import random
import time
import os
from unidecode import unidecode

import os
import zipfile
import requests
import numpy as np
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import geopy.geocoders as geocoders
import pandas as pd
import geocoder
from random import randint
import random
import time
import os
import fiona
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

def get_google_maps_url(address, proxy):
  """
  Gets the URL of a location in Google Maps.

  Args:
    address: The address of the location.

  Returns:
    The URL of the location in Google Maps.
  """

  # URL-encode the address.
  address = address.replace(" ", "%20")

  # Create the URL.
  url = "https://www.google.com/maps/search/?api=1&query=" + address

  # Make the request.
  response = requests.get(url)

  # Check the response status code.
  if response.status_code == 200:
    return response.url
  else:
    raise Exception("Error getting Google Maps URL.")



def geocoder_igac(address):
    #address = "CL 65D BIS SUR  77M  68, BOGOTÁ, D.C."
    #address = "asadadadadadadada"
    #url = get_google_maps_url(address, proxy = proxy)

  #  ##url = "https://www.google.com/maps/search/?api=1&query=Calle%2075%20a%2072"
    #req = Request(url,headers={'User-Agent': 'Mozilla/5.0'})
    #webpage = urlopen(req).read()
    ##print(webpage)

  #  #string_data = webpage.decode('utf-8')

  #  ## Extract the latitude and longitude using regular expressions
    #pattern = r"center=([\d.-]+)%2C([-\d.]+)"
    #match = re.search(pattern, string_data)

  #  #if match:
    #    latitude = float(match.group(1))
    #    longitude = float(match.group(2))
    #    #print("Latitude:", latitude)
    #    #print("Longitude:", longitude)
    #else:
    #    latitude = np.nan
    #    longitude = np.nan
    #    #print("Latitude:", latitude)
    #    #print("Longitude:", longitude)

    geocoder_arcgis = geocoder.arcgis(address)
    geojson_geocoder = geocoder_arcgis.geojson

    if geojson_geocoder["features"] == []:
        Confidence_arcgis = np.nan
        Longitude_arcgis = np.nan
        Latitude_arcgis = np.nan
    else:
        Confidence_arcgis = geojson_geocoder["features"][0]["properties"]["confidence"]
        Longitude_arcgis = geojson_geocoder["features"][0]["geometry"]["coordinates"][0]
        Latitude_arcgis = geojson_geocoder["features"][0]["geometry"]["coordinates"][1]


    # try:
    #   geolocator = geocoders.Nominatim(user_agent="my_app")
    #   location = geolocator.geocode(address)
    #       #"CALLE 74A 114 30, BOGOTÁ"
    #   if location == None:
    #       latitude_nominatim = np.nan
    #       longitude_nominatim = np.nan
    #   else:
    #       latitude_nominatim = location.latitude
    #       longitude_nominatim = location.longitude
    # except:
    #   latitude_nominatim = np.nan
    #   longitude_nominatim = np.nan
    # Initialize the geocoder


    return [address, Latitude_arcgis, Longitude_arcgis, Confidence_arcgis]

#geocoder_igac("Calle 75a N 72b - 71, Bogotá")

In [10]:
divipola = pd.read_excel("../input/CODIGOS_DIVIPOLA.xlsx", "Listado Vigentes")

divipola["Nombre Municipio"] = divipola["Nombre Municipio"].astype(str).str.strip().apply(lambda x: unidecode(x))
divipola["Nombre Departamento"] = divipola["Nombre Departamento"].astype(str).str.strip().apply(lambda x: unidecode(x))

divipola["Código Municipio"] = divipola["Código Municipio"].astype(str).str.zfill(5)
divipola["Código Departamento"] = divipola["Código Departamento"].astype(str).str.zfill(2)

In [6]:
ofertas = pd.read_excel("../input/20230717_AvaluosSucre.xlsx", "Hoja1")

Index(['Id', 'FUENTE', 'Departamento_nombre', 'Municipio_nombre', 'DIVIPOLA',
       'Zona', 'Avaluo_Fecha', 'Direccion', 'Latitud', 'Longitud',
       'Codigo_Orip', 'Matricula_Inmobiliaria', 'Condicion_Predio',
       'Tipo_de_inmueble_Inmobiliario', 'Destinacion_Economica',
       'Tipo_Unidad_Construccion', 'Anio_Construccion', 'Area_Terreno',
       'Area_Construccion', 'Garajes', 'Depositos', 'Avaluo_Terreno',
       'Avaluo_Construccion', 'Avaluo_Comercial', 'Unnamed: 24'],
      dtype='object')

In [15]:
ofertas["Municipio"] = ofertas["DIVIPOLA"].astype(str)
ofertas["Departamento"] = ofertas["DIVIPOLA"].astype(str).str[0:2]

In [16]:
#ofertas = pd.read_excel("../input/24052023_Consolidado_Avaluos.xlsx", "Hoja1")

# Function to count consecutive letters and numbers without spaces
def count_consecutive_alphanumeric(text):
    consecutive_alphanumeric = re.findall(r'[a-zA-Z0-9]+', text)
    max_length = max(len(word) for word in consecutive_alphanumeric)
    return max_length

# Function to insert spaces into the code
def insert_spaces(code):
    pattern = r'(\d+|\D+)'
    spaced_code = re.sub(pattern, r'\1 ', code).strip()
    return spaced_code

pattern = r'[^a-zA-Z0-9]'
# Assuming 'df' is your DataFrame and 'Direccion' is the column name

ofertas['Direccion1'] = ofertas['Direccion'].astype(str).str.replace('AP\s\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('APT\s\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('AP\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('APT\d+', ' ', regex = True)

ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('CS\s\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('CS\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('CASA\s\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('CASA\d+', ' ', regex = True)

ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('TO\s\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('TO\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('TORRE\s\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('TORRE\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('ETP\s\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('ETP\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('ET\s\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('ET\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('BLQ\s\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('BLQ\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('BL\s\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('BL\d+', ' ', regex = True)


ofertas.loc[(ofertas["Departamento_nombre"].isnull()) & (ofertas["Municipio"].str.endswith('000')), "Municipio"] = ofertas.loc[(ofertas["Departamento_nombre"].isnull()) & (ofertas["Municipio"].str.endswith('000')), "Municipio"].str.replace("000$", "", regex = True)
ofertas.loc[(ofertas["Departamento_nombre"].isnull()), "Departamento"] = ofertas.loc[(ofertas["Departamento_nombre"].isnull()), "Departamento"].str.zfill(2)
ofertas.loc[(ofertas["Departamento_nombre"].isnull()) & (ofertas["Municipio"].str.len() <= 3), "Municipio"] = ofertas.loc[(ofertas["Departamento_nombre"].isnull()) & (ofertas["Municipio"].str.len() <= 3), "Municipio"].str.zfill(3)
ofertas.loc[(ofertas["Departamento_nombre"].isnull()) & (ofertas["Municipio"].str.len() <= 3), "Municipio"] = ofertas.loc[(ofertas["Departamento_nombre"].isnull()) & (ofertas["Municipio"].str.len() <= 3), "Departamento"].astype(str) + ofertas.loc[(ofertas["Departamento_nombre"].isnull()) & (ofertas["Municipio"].str.len() <= 3), "Municipio"].astype(str)
ofertas["Municipio"] = ofertas["Municipio"].astype(str)
ofertas["Departamento"] = ofertas["Departamento"].astype(str)

# Replace the cases with four or more consecutive zeroes with a different value
ofertas['Direccion1'] = ofertas['Direccion1'].str.replace(r'\d{5,}', '')

ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('BL\s\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('BL\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('BLOQUE\s\d+', ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace('BLOQUE\d+', ' ', regex = True)

ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.extract(r'(\D*\d+\D*\d+\D*\d+)')
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace(pattern, ' ', regex = True)
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace(" NO ", ' ')
ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.replace(' N ', ' ')

print(ofertas.shape)
ofertas = pd.merge(ofertas, divipola[["Código Municipio", "Nombre Municipio", "Nombre Departamento"]].drop_duplicates(), how = "left", left_on = ["Municipio"], right_on = ["Código Municipio"])
print(ofertas.shape)

ofertas.loc[(ofertas["Departamento_nombre"].isnull()), "Municipio_nombre"] = ofertas.loc[(ofertas["Departamento_nombre"].isnull()), "Nombre Municipio"]
ofertas.loc[(ofertas["Departamento_nombre"].isnull()), "Departamento_nombre"] = ofertas.loc[(ofertas["Departamento_nombre"].isnull()), "Nombre Departamento"]

ofertas['Direccion1'] = ofertas['Direccion1'].astype(str).str.strip().apply(lambda x: unidecode(x))
ofertas["Direccion1"] = ofertas["Direccion1"].apply(insert_spaces)
ofertas['Direccion2'] = ofertas['Direccion1'].astype(str) + ", " +ofertas['Municipio_nombre'].astype(str) + ", " + ofertas["Departamento_nombre"].astype(str)
ofertas['Direccion2'] = ofertas['Direccion2'].astype(str).str.replace(', BOGOTÁ, D.C., BOGOTÁ, D.C.', ', BOGOTÁ D.C.')
ofertas['Direccion2'] = ofertas['Direccion2'].astype(str).str.replace(', BOGOTA, D.C., BOGOTA, D.C.', ', BOGOTA D.C.')
ofertas['Direccion2'] = ofertas['Direccion2'].str.upper().str.strip()
ofertas['Direccion2'] = ofertas['Direccion2'].astype(str).str.strip().apply(lambda x: unidecode(x))
ofertas['Direccion2'] = ofertas['Direccion2'].astype(str).str.replace(' , ', ', ')
ofertas['Direccion2'] = ofertas['Direccion2'].astype(str).str.replace('\t', '')
ofertas['Direccion2'] = ofertas['Direccion2'].astype(str).apply(lambda x: re.sub(r'\s{2,}', ' ', x.strip()))



(3736, 28)
(3736, 31)


In [18]:
list_directions = ofertas['Direccion2'].tolist()
sorted_list = sorted(map(str, list_directions))

In [22]:
my_min = 0
my_max = len(sorted_list)#25000

indexes = list(range(my_min, my_max))
my_list = sorted_list

# Define the chunk size
chunk_size = 5000
df = []
# Iterate over the list and apply the function
for i, element in enumerate(sorted_list):
    result = geocoder_igac(element)
    result = result + [indexes[i]]
    
    df.append(result)
     # Save the results to a CSV file every chunk_size elements
    if (i % chunk_size == 0 or i == (len(my_list)-1)) & (i != 0):
        # Create a DataFrame with the results
        #df = pd.DataFrame(result, columns=['Result'])       
        
        # Generate the CSV file name
        file_name = f'../output/Resultados_Georref_Sucre.csv'

        # Save the DataFrame to a CSV file
        pd.DataFrame(df).to_csv(file_name, index=False)

In [23]:

# Generate the CSV file name
file_name = f'output/Results_{i}.csv'

# Save the DataFrame to a CSV file
pd.DataFrame(df).to_csv(file_name, index=False)

In [22]:
pd.DataFrame(df).shape

(25000, 6)